In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from transformers import AutoTokenizer, XLMRobertaModel, Trainer, TrainingArguments, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import wandb

In [ ]:
# Load the datasets
df1 = pd.read_excel("/content/mizo_sentiment_dataset.xlsx")
df2 = pd.read_excel("/content/Chungli_ao_Train.xlsx")

In [ ]:
#df = df1
df = pd.concat([df1, df2])

In [ ]:
# Display information about the dataframe
df.info()

In [ ]:
# Converting the labels to numbers
le = preprocessing.LabelEncoder()
df['Sentiment'] = le.fit_transform(df.Sentiment.values)

In [ ]:
# Shuffle the dataframe
df = shuffle(df)

In [ ]:
# Count the number of positive and negative labels
Positive_labels = (df['Sentiment'] == 1).sum()
Negative_labels = (df['Sentiment'] == 0).sum()
print(f"Number of Postive labels: {Positive_labels}")
print(f"Number of Negative labels: {Negative_labels}")

In [ ]:
# Split the data into training and validation sets
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [ ]:
# Load MizBERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("robzchhangte/MizBERT")

In [ ]:
# Load XLM-Roberta model
model_name = 'xlm-roberta-base'
model = XLMRobertaModel.from_pretrained(model_name)

In [ ]:
# Ensure all elements are strings
train_texts = [str(text) for text in train_texts]
val_texts = [str(text) for text in val_texts]

In [ ]:
# Tokenize the texts with MizBERT tokenizer (truncation, padding, max_length of 512)
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# Define custom dataset class
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Encode labels
train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

In [ ]:
# Create dataset objects
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)

In [ ]:
# Define custom classification model using XLM-Roberta model
class CustomClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(CustomClassifier, self).__init__()
        self.base_model = XLMRobertaModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)  # Removed token_type_ids
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the representation of [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [ ]:
# Instantiate the custom model with XLM-Roberta base
custom_model = CustomClassifier(model_name='xlm-roberta-base', num_labels=2)

In [ ]:
# Define metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc, 'F1': f1, 'Precision': precision, 'Recall': recall}

In [ ]:
!pip install wandb
import wandb

In [ ]:
# Initialize wandb
wandb.init(project='Roboust_nlp_XLMR', name='Chungliao+Mizo_mizbert_tokenizer_second_run')

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    learning_rate=0.0001,
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,   # batch size per device during training
    per_device_eval_batch_size=16,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,              # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10,
    report_to='wandb',
)

# Initialize the Trainer
trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # compute metrics function
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)]
)

In [ ]:
# Train the model
trainer.train()

In [1]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

NameError: name 'trainer' is not defined

#Test_set

In [ ]:
test_df = pd.read_excel("/content/Chungli_Ao_test.xlsx")

In [ ]:
len(test_df)

In [ ]:
test_df.head()

In [ ]:
test_df.tail()

In [ ]:
test_texts = list(test_df['Text'])

In [ ]:
test_df['Sentiment'] = le.fit_transform(test_df.Sentiment.values)

In [ ]:
test_labels = test_df['Sentiment']

In [ ]:
len(test_labels)

In [ ]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = CustomDataset(test_encodings, test_labels)

In [ ]:
test_metrics = trainer.evaluate(test_dataset)

In [ ]:
print(test_metrics)